# **Swyp Teddy Bear Club NFT Rating and Rewards Calculation**

## **1. Overview**

### **1.1 Purpose**
This document outlines the process for calculating the rankings and rewards for the Teddy Bear Club NFTs across Round 1 and Round 2. Since the ranking and reward structures differ between the two rounds, calculations are performed separately for each round and then combined to determine the total rewards, where applicable.

### **1.2 NFT Information**
Round 1:<br>
   - Policy Id: ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87 <br>
   - Total NFTs: 804<br>
   - Tier System:<br>
      - Tier 1 - Rarity 1 - 15 : 28,000 TEDY
      - Tier 2 - Rarity 16 - 100 : 17,500 TEDY
      - Tier 3 - Rarity 101 - 250 : 12,600 TEDY
      - Tier 4 - Rarity 251 - 500 : 11,200 TEDY
      - Tier 5 - Rarity 501 - 804 : 10,500 TEDY
      

Round 2:<br>
   - Policy Id: da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c<br>
   - Total NFTs: 2,035<br>
   - Tier System:<br>
      - Tier 1 - Rarity 1 - 23 : 7,000 TEDY
      - Tier 2 - Rarity 24 - 221 : 6,000 TEDY
      - Tier 3 - Rarity 222 - 664 : 5,000 TEDY
      - Tier 4 - Rarity 665 - 1,327 : 4,500 TEDY
      - Tier 5 - Rarity 1,328 - 2,035 : 4,200 TEDY

---

### **1.3 Workflow Summary**
The workflow is divided into the following steps:

1. **Fetch Data**  
   - Load NFT data from JSON files.

2. **NFT Rank Calculation**  
   2.1 Assign point values based on trait rarity.  
   2.2 Compute overall rarity ranks for each NFT.

3. **NFT Rewards Calculation**  
   - Calculate rewards for each NFT based on its rank and the round-specific reward structure.

---

### **1.4 Necessary Package Installations and Open Statements**


In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json"
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"

#r "nuget: Plotly.NET.Interactive, 5.0.0"

In [27]:
open System.Text.Json
open System.IO
open FSharp.Reflection
open Plotly.NET
open Plotly.NET.Interactive

# **2. Workflow**

## 1. Fetch Data
The metadata for the NFTs was scraped from the web and stored in JSON files. In this section, we will parse the JSON data to extract information about the NFTs and the occurrence of their traits. This data will serve as the foundation for calculating rankings and determining rewards in the subsequent sections.

---

**Types**:
The NFT and NFTTraits types are partial representations of the data, as only the fields relevant to ranking and reward calculations have been included.

- **NFT**: Represents an individual NFT, including its metadata and traits.

- **NFTTraits**: Captures occurrence counts of trait values across NFTs.

**Function**:
- **loadNFTs<'T>**: Loads and deserializes JSON data into a specified type:
    - Parameters: 
        - filePath (string) – the path to the JSON file.
    - Returns: 'T 

**Variables**:
- **nftRound1NFTS**: NFT list – NFTs from Round 1
- **nftRound2NFTS**: NFT list – NFTs from Round 2
- **nftRound1Traits**: NFTTraits – Trait counts for Round 1
- **nftRound2Traits**: NFTTraits – Trait counts for Round 2


In [112]:
type NFT = {
    assetName: string
    encodedName: string
    Background: string
    Bear: string
    Clothes: string
    Face: string
    Handheld: string
    Head: string
    Skins: string
}

type NFTTraits = {
    Background: Map<string, int>
    Bear: Map<string, int>
    Clothes: Map<string, int>
    Face: Map<string, int>
    Handheld: Map<string, int>
    Head: Map<string, int>
    Skins: Map<string, int>
}

In [113]:
let loadNFTs<'T> (filePath: string) : 'T =
    let jsonContent = File.ReadAllText(filePath)
    JsonSerializer.Deserialize<'T>(jsonContent)

In [114]:
let nftRound1NFTS = loadNFTs<NFT list> "./NFTR1.json"
let nftRound2NFTS = loadNFTs<NFT list> "./NFTR2.json"
let nftRound1Data = loadNFTs<NFTTraits list> "./JsonScraper/NFT1Traits.json"
let nftRound2Data = loadNFTs<NFTTraits list> "./JsonScraper/NFT2Traits.json"
let nftRound1Traits = List.head nftRound1Data
let nftRound2Traits = List.head nftRound2Data

## **2. NFT Rank Calculation** 
To calculate the points for each trait, we first divide the number of occurrences of that trait by the total number of NFTs in the round, which gives us the percentage of NFTs that have that trait. We then subtract this percentage from 100, creating a direct relationship between rarity and points so rarer traits get higher points. Finally, we sum the points for each trait across different attributes (such as Background, Bear, Clothes, etc.) to calculate the final rarity score for each NFT.

$\text{Trait Score} = \text{round}\left(100.0 - \left(\frac{\text{value}}{\text{totalNfts}}\right) \cdot 100.0, 2\right)$

$\text{Total Rarity Score} = \sum_{i=1}^{7} T_i$

---
### 2.1 Assign point values based on trait rarity.   

**Function**:
- *processNFTTraits*: Calculates rarity scores for each trait dynamically for a given NFTTraits record and total NFTs in the round.
    - *Parameters*:
        *nftTraits*: NFTTraits – The trait occurrence data for a round.
    - *totalNfts*: float – The total number of NFTs in the round.
    - Returns: Map<string, Map<string, float>>

**Variables**:
- *r1TotalNfts*: float – The total number of NFTs in Round 1 (804).
- *r2TotalNfts*: float – The total number of NFTs in Round 2 (2035).
- *r1ScoredTraits*: Map<string, Map<string, float>> – A map of scored trait percentages for Round 1 NFTs.
- *r2ScoredTraits*: Map<string, Map<string, float>> – A map of scored trait percentages for Round 2 NFTs.


In [115]:
let r1TotalNfts = 804.0
let r2TotalNfts = 2035.0

In [116]:
let processNFTTraits (nftTraits: NFTTraits) (totalNfts: float) =
    let recordType = typeof<NFTTraits>
    FSharp.Reflection.FSharpType.GetRecordFields(recordType)
    |> Array.fold (fun acc field ->
        let fieldValue = field.GetValue(nftTraits) :?> Map<string, int>
        let traitScores = 
            fieldValue |> Map.map (fun key value ->
                Math.Round(100.0 - (float value / float totalNfts) * 100.0, 2)
            )
        Map.add field.Name traitScores acc
    ) Map.empty

In [117]:
let r1ScoredTraits = processNFTTraits nftRound1Traits r1TotalNfts
let r2ScoredTraits = processNFTTraits nftRound2Traits r2TotalNfts

### *Illustrating the Distribution of Trait Scores*: 
This section visualizes the percentages of trait occurrences alongside their corresponding scores. The createCharts function processes the trait data and generates doughnut charts that display both the raw values and their proportions.

In [123]:
let createCharts (TraitScores: Map<string, Map<string, float>>) = 
    TraitScores
    |> Map.fold( fun acc title labelsAndValues ->
        let labels,values = 
            labelsAndValues
            |> Map.toList
            |> List.unzip

        let combinedText = 
            List.map2 (fun label value -> sprintf "%s: %.2f" label value) labels values

        let chart = 
            Chart.Doughnut(values = values, Labels = labels, Hole = 0.5, MultiText = combinedText)
            |> Chart.withTitle title
            |> Chart.withLayout(
                Layout.init(
                    Width = 1200,
                    Height = 1000
                )
            )
                    
        chart :: acc
    )[]
    |> List.rev

let chartRound1 = createCharts r1ScoredTraits 
let chartRound2 = createCharts r2ScoredTraits

### Teddy Bear Club Round 1 Trait Scores

In [129]:
for chart in chartRound1 do
    display(chart |> Chart.withSize(1400,1200))

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

### Teddy Bears Club Round 2 Trait Scores

In [ ]:
for chart in chartRound2 do
    display(chart |> Chart.) 

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

<!-- Plotly chart will be drawn inside this DIV -->

### 2.2 Compute overall rarity ranks for each NFT. 
Using the information from this step, we also created a CSV mapping NFTs to their ranks for easier access and computation.

**Function**:
- *computeNFTRank*: Computes the rarity rank of NFTs based on their traits and scores.
    - *Parameters*:
        *nftList*: NFT list – A list of NFTs to rank.
        *nftTraitScores*: Map<string, Map<string, float>> – Trait scores computed above.

**Variables**:
- *rankedNFTsRound1*: Ranked list of Round 1 NFTs as (rank, assetName, encodedName, rarityScore).
- *rankedNFTsRound2*: Ranked list of Round 2 NFTs with the same structure as above.


In [121]:
let computeNFTRank (nftList: list<NFT>) nftTraitScores = 
    nftList
    |> List.map (fun nft ->
        let rarityScore =  
            [
                ("Background", nft.Background)
                ("Bear", nft.Bear)
                ("Clothes", nft.Clothes)
                ("Face", nft.Face)
                ("Handheld", nft.Handheld)
                ("Head", nft.Head)
                ("Skins", nft.Skins)
            ]
            |> List.fold (fun acc (attrName, attrValue) ->  
                match Map.tryFind attrName nftTraitScores with
                | Some traitMap ->  
                    acc + (Map.tryFind attrValue traitMap |> Option.defaultValue 0.0)
                | None -> acc 
            ) 0.0 
        (nft.assetName, nft.encodedName, rarityScore) 
    )
    |> List.sortBy (fun (_, _, rarityScore) -> -rarityScore)
    |> List.mapi (fun index (assetName, encodedName, rarityScore) -> 
    (index + 1, assetName, encodedName, rarityScore))

In [122]:
let rankedNFTsRound1 = computeNFTRank nftRound1NFTS r1ScoredTraits
let rankedNFTsRound2 = computeNFTRank nftRound2NFTS r2ScoredTraits

## **3. NFT Rewards Calculation** 
To calculate the points for each trait, we first divide the number of occurrences of that trait by the total number of NFTs in the round, which gives us the percentage of NFTs that have that trait. We then subtract this percentage from 100, creating a direct relationship between rarity and points so rarer traits get higher points. Finally, we sum the points for each trait across different attributes (such as Background, Bear, Clothes, etc.) to calculate the final rarity score for each NFT.

$\text{Trait Score} = \text{round}\left(100.0 - \left(\frac{\text{value}}{\text{totalNfts}}\right) \cdot 100.0, 2\right)$

$\text{Total Rarity Score} = \sum_{i=1}^{7} T_i$

---

In [1]:
//Parse CSV
let parseRankData (filePath: string) =
    File.ReadAllLines(filePath)
    |> Array.skip 1
    |> Array.map (fun line ->
        let parts = line.Split(',')
        let subject = parts.[0]
        let rank = int parts.[1]
        subject, rank
    )
    |> Map.ofArray

let subjectRankData = parseRankData "NFT_Ranks.csv"

In [ ]:
let round1PolicyId = "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87"
let round2PolicyId = "da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c"

let categorizeNFTRound1ByRank nft rank (nftsCount: int array)=
    printfn "Calculating rewards for NFT1: %s with rank %d" nft rank
    if rank <= 15 then nftsCount[0] <- nftsCount[0] + 1
    elif rank <= 100 then nftsCount[1] <- nftsCount[1]+ 1
    elif rank <= 250 then nftsCount[2] <- nftsCount[2] + 1
    elif rank <= 500 then nftsCount[3] <- nftsCount[3] + 1
    elif rank <= 804 then nftsCount[4] <- nftsCount[4] + 1
    else printfn "Rank %d for NFT %s does not fall within any predefined range" rank nft

let categorizeNFTRound2ByRank nft rank (nftsCount: int array)=
    printfn "Calculating rewards for NFT2: %s with rank %d" nft rank
    if rank <= 23 then nftsCount[0] <- nftsCount[0] + 1
    elif rank <= 221 then nftsCount[1] <- nftsCount[1] + 1
    elif rank <= 664 then nftsCount[2] <- nftsCount[2] + 1
    elif rank <= 1327 then nftsCount[3] <- nftsCount[3] + 1
    elif rank <= 2035 then nftsCount[4] <- nftsCount[4] + 1
    else printfn "Rank %d for NFT %s does not fall within any predefined range" rank nft

let countNFTsPerRound (ownedNFTs: list<string>) (rankData: Map<string, int>) =
    let round1NFTsHeldByRank = [|0;0;0;0;0|]
    let round2NFTsHeldByRank = [|0;0;0;0;0|]
    ownedNFTs
    |> List.iter (fun nft ->
        match rankData.TryFind nft with
        | Some rank ->
            if nft.Contains(round1PolicyId) then
                categorizeNFTRound1ByRank nft rank round1NFTsHeldByRank
            elif nft.Contains(round2PolicyId) then
                categorizeNFTRound2ByRank nft rank round2NFTsHeldByRank
            else
                printfn "Unknown NFT policyId for %s" nft 
        | None ->
            printfn "Rank data not found for NFT: %s" nft
    )
    (round1NFTsHeldByRank, round2NFTsHeldByRank)
    

In [16]:
//test
let ownedNFTs = [
                    "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d8754656464794265617273436c7562313738";
                    "da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c54656464794265617273436c756234373930"
                ]

let round1NFTsHeldByRank, round2NFTsHeldByRank = countNFTsPerRound ownedNFTs subjectRankData

printfn "Round 1 Rewards: %A" round1NFTsHeldByRank
printfn "Round 2 Rewards: %A" round2NFTsHeldByRank


Calculating rewards for NFT1: ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d8754656464794265617273436c7562313738 with rank 1
Calculating rewards for NFT2: da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c54656464794265617273436c756234373930 with rank 94
Round 1 Rewards: [|1; 0; 0; 0; 0|]
Round 2 Rewards: [|0; 1; 0; 0; 0|]


In [ ]:
let round1Bonus = 2.75
let round1NFTsSold = 804
let round2NFTsSold = 2035

let calculatePercentShareFromAvailableRewards = 
    let round1NFTsHeld = float (Array.sum round1NFTsHeldByRank)
    let round2NFTsHeld = float (Array.sum round2NFTsHeldByRank)
    let totalNFTsSold = (float round1NFTsSold * round1Bonus + float round2NFTsSold)

    let round1PercentShare = round1NFTsHeld * round1Bonus / totalNFTsSold * 100.0
    let round2PercentShare = round2NFTsHeld / totalNFTsSold * 100.0
    (round1PercentShare,round2PercentShare)

let round1PercentShare, round2PercentShare = calculatePercentShareFromAvailableRewards

In [43]:
let round1Tier1 = 30800
let round1Tier2 = 19250
let round1Tier3 = 13860
let round1Tier4 = 12320
let round1Tier5 = 11550

let round2Tier1 = 7000
let round2Tier2 = 6000
let round2Tier3 = 5000
let round2Tier4 = 4500
let round2Tier5 = 4200

let countOfTier1 = 23
let countOfTier2 = 198
let countOfTier3 = 443
let countOfTier4 = 663
let countOfTier5 = 708

let maxRoundTwoTedyTokens = 42000000
let distributedTokens = (countOfTier1 * round2Tier1) + (countOfTier2 * round2Tier2) + (countOfTier3 * round2Tier3) + (countOfTier4 * round2Tier4) + (countOfTier5 * round2Tier5)
let availableTokenRewards = float (maxRoundTwoTedyTokens - distributedTokens)

let calculateApproximateTbcRewards = 
    let round1Share = round1PercentShare / 100.0
    let round2Share = round2PercentShare / 100.0

    let round1ApproximateTedyShare = Math.Ceiling(availableTokenRewards * round1Share)
    let round2ApproximateTedyShare = Math.Ceiling(availableTokenRewards * round2Share)

    ((int)round1ApproximateTedyShare,(int)round2ApproximateTedyShare)

let round1TedyShare,round2TedyShare = calculateApproximateTbcRewards

In [44]:
//total rewards
//compute rewards per round (rank by amount)
let calculateTedyTokensTotal = 
    let round1TotalTedy = (round1NFTsHeldByRank[0] * round1Tier1) + (round1NFTsHeldByRank[1] * round1Tier2) + (round1NFTsHeldByRank[2] * round1Tier3) + (round1NFTsHeldByRank[3] * round1Tier4) + (round1NFTsHeldByRank[4] * round1Tier5)
    let round2TotalTedy = (round2NFTsHeldByRank[0] * round2Tier1) + (round2NFTsHeldByRank[1] * round2Tier2) + (round2NFTsHeldByRank[2] * round2Tier3) + (round2NFTsHeldByRank[3] * round2Tier4) + (round2NFTsHeldByRank[4] * round2Tier5)

    let round1TotalTedyTotal = round1TotalTedy + round1TedyShare
    let round2TotalTedyTotal = round2TotalTedy + round2TedyShare
    (round1TotalTedyTotal,round2TotalTedyTotal)

let round1ApproximateTotalTedy,round2ApproximateTotalTedy = calculateTedyTokensTotal

In [52]:
//Code to calculate Marketcap Adjustment to 8,000,000 supply from 100 000 000

let marketcapAdjustment (rewards: float) = 
    let finalRoundReward = (rewards*8000000.0)/100000000.0
    Math.Ceiling(finalRoundReward)

let totalTedyRound1 = int(marketcapAdjustment round1ApproximateTotalTedy)
let totalTedyRound2 = int(marketcapAdjustment round2ApproximateTotalTedy)
let totalTedyReward = totalTedyRound1 + totalTedyRound2

printfn "%d" totalTedyRound1
printfn "%d" totalTedyRound2
printfn "%d" totalTedyReward


4147
1092
5239
